This is my final for my Machine Learning class.  In the end I should have picked a easier project. When I selected to try and compete in the competition I had high hopes that I would get an A and $25,000. Now, I am ready to settle for a C and probably never submit a sample to get checked. I had been working on the competition for a week before I started doing it as a school project and I really had no clue what I was doing but looking at other kernels got me started and honestly carried me through this. Big shout out to Radu Stoicescu, he has an amazing kernel that uses blob detection to find coordinates of the sea lions. I used his methods and  then used a classifier I had used in class. I give little sections throughout on what/ why I did things. If you are someone besides my Professor feel free to look through and see what I did, it might help. Comment if you really want to. Wasn't the best project but I had a good time and can say I tried.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input/"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import cv2
import matplotlib.pyplot as plt
import skimage.feature
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, Cropping2D
from keras.utils import np_utils
import random
from random import randint
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import cross_val_predict
import pylab as pl
import seaborn as sn
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.neural_network import MLPClassifier
from collections import Counter
from sklearn.metrics import roc_auc_score


%matplotlib inline

Displaying Pictures of training and training dotted, then a zoomed in version to give a better view

In [ ]:
train_data = pd.read_csv('../input/Train/train.csv')
train_imgs = sorted(glob.glob('../input/Train/*.jpg'), key=lambda name: int(os.path.basename(name)[:-4]))
train_dot_imgs = sorted(glob.glob('../input/TrainDotted/*.jpg'), key=lambda name: int(os.path.basename(name)[:-4]))
index = 0

sl_counts = train_data.iloc[index]
print(sl_counts)

print(train_imgs[index])
img = cv2.cvtColor(cv2.imread(train_imgs[index]), cv2.COLOR_BGR2RGB)
img_dot = cv2.cvtColor(cv2.imread(train_dot_imgs[index]), cv2.COLOR_BGR2RGB)

f, ax = plt.subplots(1,2,figsize=(16,8))
(ax1, ax2) = ax.flatten()

ax1.imshow(img)
ax2.imshow(img_dot)

In [ ]:
crop_img = img[1350:1900, 3000:3400]
crop_img_dot = img_dot[1350:1900, 3000:3400]

f, ax = plt.subplots(1,2,figsize=(16,8))
(ax1, ax2) = ax.flatten()

ax1.imshow(crop_img)
ax2.imshow(crop_img_dot)

plt.show()

Histograms to show the distribution of data within the entire dataset and then of the first picture which was the main picture used to create a training set.

In [ ]:
hist = train_data.sum(axis=0)
print(hist)


sea_lions_types = hist[1:]
f, ax1 = plt.subplots(1,1,figsize=(5,5))
sea_lions_types.plot(kind='bar', title='Count of Sea Lion Types (Train)', ax=ax1)
plt.show()

In [ ]:
index = 0
sl_counts = train_data.iloc[index]
print(sl_counts)

plt.figure()
sl_counts.plot(kind='bar', title='Count of Sea Lion Types')
plt.show()

Radu Stoicescu Blob detection method. The only major thing that I changed was the sample size, I used the first, second, and third picture. But honestly it wouldn't have made that much of a difference. As shown in the histogram below that the second and third photos don't contribute much to the sea lions, but they add about 100 more 'negative' photos each, which is important for making a model.

In [ ]:
# CREDITS GO TO:  Radu Stoicescu
class_names = ['adult_females', 'adult_males', 'juveniles', 'pups', 'subadult_males']

file_names = os.listdir("../input/Train/")
file_names = sorted(file_names, key=lambda 
                    item: (int(item.partition('.')[0]) if item[0].isdigit() else float('inf'), item)) 

# select a subset of files to run on
file_names = file_names[0:3]
print(file_names)
# dataframe to store results in
coordinates_df = pd.DataFrame(index=file_names, columns=class_names)

In [ ]:
# CREDITS GO TO:  Radu Stoicescu
for filename in file_names:
    
    # read the Train and Train Dotted images
    image_1 = cv2.imread("../input/TrainDotted/" + filename)
    image_2 = cv2.imread("../input/Train/" + filename)
    
    cut = np.copy(image_2)
    
    # absolute difference between Train and Train Dotted
    image_3 = cv2.absdiff(image_1,image_2)
    
    # mask out blackened regions from Train Dotted
    mask_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    mask_1[mask_1 < 20] = 0
    mask_1[mask_1 > 0] = 255
    
    mask_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)
    mask_2[mask_2 < 20] = 0
    mask_2[mask_2 > 0] = 255
    
    image_3 = cv2.bitwise_or(image_3, image_3, mask=mask_1)
    image_3 = cv2.bitwise_or(image_3, image_3, mask=mask_2) 
    
    # convert to grayscale to be accepted by skimage.feature.blob_log
    image_3 = cv2.cvtColor(image_3, cv2.COLOR_BGR2GRAY)
    
    # detect blobs
    blobs = skimage.feature.blob_log(image_3, min_sigma=3, max_sigma=4, num_sigma=1, threshold=0.02)
    
    adult_males = []
    subadult_males = []
    pups = []
    juveniles = []
    adult_females = [] 
    
    image_circles = image_1
    
    for blob in blobs:
        # get the coordinates for each blob
        y, x, s = blob
        # get the color of the pixel from Train Dotted in the center of the blob
        g,b,r = image_1[int(y)][int(x)][:]
        
        # decision tree to pick the class of the blob by looking at the color in Train Dotted
        if r > 200 and g < 50 and b < 50: # RED
            adult_males.append((int(x),int(y)))
            cv2.circle(image_circles, (int(x),int(y)), 20, (0,0,255), 10) 
        elif r > 200 and g > 200 and b < 50: # MAGENTA
            subadult_males.append((int(x),int(y))) 
            cv2.circle(image_circles, (int(x),int(y)), 20, (250,10,250), 10)
        elif r < 100 and g < 100 and 150 < b < 200: # GREEN
            pups.append((int(x),int(y)))
            cv2.circle(image_circles, (int(x),int(y)), 20, (20,180,35), 10)
        elif r < 100 and  100 < g and b < 100: # BLUE
            juveniles.append((int(x),int(y))) 
            cv2.circle(image_circles, (int(x),int(y)), 20, (180,60,30), 10)
        elif r < 150 and g < 50 and b < 100:  # BROWN
            adult_females.append((int(x),int(y)))
            cv2.circle(image_circles, (int(x),int(y)), 20, (0,42,84), 10)  
            
        cv2.rectangle(cut, (int(x)-112,int(y)-112),(int(x)+112,int(y)+112), 0,-1)
            
    coordinates_df["adult_males"][filename] = adult_males
    coordinates_df["subadult_males"][filename] = subadult_males
    coordinates_df["adult_females"][filename] = adult_females
    coordinates_df["juveniles"][filename] = juveniles
    coordinates_df["pups"][filename] = pups

In [ ]:
f, ax = plt.subplots(1,1,figsize=(10,16))
ax.imshow(cv2.cvtColor(image_circles, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
f, ax = plt.subplots(1,1,figsize=(10,16))
ax.imshow(cv2.cvtColor(cut, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# CREDITS GO TO:  Radu Stoicescu
x = []
y = []

for filename in file_names:    
    image = cv2.imread("../input/Train/" + filename)
    for lion_class in class_names:
        for coordinates in coordinates_df[lion_class][filename]:
            thumb = image[coordinates[1]-32:coordinates[1]+32,coordinates[0]-32:coordinates[0]+32,:]
            if np.shape(thumb) == (64, 64, 3):
                x.append(thumb)
                y.append(lion_class)

In [ ]:
# CREDITS GO TO:  Radu Stoicescu
for i in range(0,np.shape(cut)[0],224):
    for j in range(0,np.shape(cut)[1],224):                
        thumb = cut[i:i+64,j:j+64,:]
        if np.amin(cv2.cvtColor(thumb, cv2.COLOR_BGR2GRAY)) != 0:
            if np.shape(thumb) == (64,64,3):
                x.append(thumb)
                y.append("negative")  

In [ ]:
# CREDITS GO TO:  Radu Stoicescu
class_names.append("negative")
y.count('negative')

In previous cell it shows the count of negative photo count to be 337.  It is important to have a lot of negatives and in the future I feel it would be better to add more. Histogram and Table showing sea lion counts and distribution.

In [ ]:
sl_counts = train_data.iloc[0:3]
print(sl_counts)

plt.figure()
sl_counts.plot(kind='bar', title='Count of Sea Lion Types')
plt.show()

In [ ]:
# CREDITS GO TO:  Radu Stoicescu
x = np.array(x)
y = np.array(y)

I did this next cell, the purpose was that the multi layer classifier I used doesn't accept anything over 2D so I converted from (1368,64,64,3) to (1368,12288) small but very important change for what I did.

In [ ]:
z = []
for img in x:
    img = np.array(img).reshape(12288)
    z.append(img)
z = np.array(z)
x.shape

Really cool function that displays the first 10 pictures of the classes. But I think it has some insight into why the model may be very inaccurate when there are females present but no pups. In the pups row pictures 7 and 9 are the exact same. But in 7 the picture is focused on the pup and in 9 the picture is focused on the female. I believe that this is causing the model to confuse females and pups and misidentify them. 

In [ ]:
# CREDITS GO TO:  Radu Stoicescu
for lion_class in class_names:
    f, ax = plt.subplots(1,10,figsize=(12,1.5))
    f.suptitle(lion_class)
    axes = ax.flatten()
    j = 0
    for a in axes:
        a.set_xticks([])
        a.set_yticks([])
        for i in range(j,len(x)):
            if y[i] == lion_class:
                j = i+1
                a.imshow(cv2.cvtColor(x[i], cv2.COLOR_BGR2RGB))
                break

In [ ]:
encoder = LabelBinarizer()
encoder.fit(y)
y = encoder.transform(y).astype(float)

Initiating the Multilayer classifier. And yes it would be a much better idea to use a neural network but I wanted to try and get the classifier to work since I'm classifying sea lions. The model can pretty easily get to around 90% accuracy but above that is hard to attain. And for my purposes it works. 

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(100, 100), random_state=42)

In [ ]:
clf.fit(z,y)

Now I create my test set. I did it in the exact same fashion as the training set. But the key difference was that I only did one picture at a time. Again this is not even close to what the competition would call for because I'm extracting images from the training images and not iterating over the test images and counting what sea lions I see. But this works with what I wanted to accomplish.

In [ ]:
# CREDITS GO TO:  Radu Stoicescu
class_names = ['adult_females', 'adult_males', 'juveniles', 'pups', 'subadult_males']

file_names = os.listdir("../input/Train/")
file_names = sorted(file_names, key=lambda 
                    item: (int(item.partition('.')[0]) if item[0].isdigit() else float('inf'), item)) 

# select a subset of files to run on
file_names = file_names[8:9]

# dataframe to store results in
coordinates_df = pd.DataFrame(index=file_names, columns=class_names)


for filename in file_names:
    
    # read the Train and Train Dotted images
    image_1 = cv2.imread("../input/TrainDotted/" + filename)
    image_2 = cv2.imread("../input/Train/" + filename)
    
    cut = np.copy(image_2)
    
    # absolute difference between Train and Train Dotted
    image_3 = cv2.absdiff(image_1,image_2)
    
    # mask out blackened regions from Train Dotted
    mask_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    mask_1[mask_1 < 20] = 0
    mask_1[mask_1 > 0] = 255
    
    mask_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)
    mask_2[mask_2 < 20] = 0
    mask_2[mask_2 > 0] = 255
    
    image_3 = cv2.bitwise_or(image_3, image_3, mask=mask_1)
    image_3 = cv2.bitwise_or(image_3, image_3, mask=mask_2) 
    
    # convert to grayscale to be accepted by skimage.feature.blob_log
    image_3 = cv2.cvtColor(image_3, cv2.COLOR_BGR2GRAY)
    
    # detect blobs
    blobs = skimage.feature.blob_log(image_3, min_sigma=3, max_sigma=4, num_sigma=1, threshold=0.02)
    
    adult_males = []
    subadult_males = []
    pups = []
    juveniles = []
    adult_females = [] 
    
    image_circles = image_1
    
    for blob in blobs:
        # get the coordinates for each blob
        y, x, s = blob
        # get the color of the pixel from Train Dotted in the center of the blob
        g,b,r = image_1[int(y)][int(x)][:]
        
        # decision tree to pick the class of the blob by looking at the color in Train Dotted
        if r > 200 and g < 50 and b < 50: # RED
            adult_males.append((int(x),int(y)))
            cv2.circle(image_circles, (int(x),int(y)), 20, (0,0,255), 10) 
        elif r > 200 and g > 200 and b < 50: # MAGENTA
            subadult_males.append((int(x),int(y))) 
            cv2.circle(image_circles, (int(x),int(y)), 20, (250,10,250), 10)
        elif r < 100 and g < 100 and 150 < b < 200: # GREEN
            pups.append((int(x),int(y)))
            cv2.circle(image_circles, (int(x),int(y)), 20, (20,180,35), 10)
        elif r < 100 and  100 < g and b < 100: # BLUE
            juveniles.append((int(x),int(y))) 
            cv2.circle(image_circles, (int(x),int(y)), 20, (180,60,30), 10)
        elif r < 150 and g < 50 and b < 100:  # BROWN
            adult_females.append((int(x),int(y)))
            cv2.circle(image_circles, (int(x),int(y)), 20, (0,42,84), 10)  
            
        cv2.rectangle(cut, (int(x)-112,int(y)-112),(int(x)+112,int(y)+112), 0,-1)
            
    coordinates_df["adult_males"][filename] = adult_males
    coordinates_df["subadult_males"][filename] = subadult_males
    coordinates_df["adult_females"][filename] = adult_females
    coordinates_df["juveniles"][filename] = juveniles
    coordinates_df["pups"][filename] = pups



x_test = []
y_test = []

for filename in file_names:    
    image = cv2.imread("../input/Train/" + filename)
    for lion_class in class_names:
        for coordinates in coordinates_df[lion_class][filename]:
            thumb = image[coordinates[1]-32:coordinates[1]+32,coordinates[0]-32:coordinates[0]+32,:]
            if np.shape(thumb) == (64, 64, 3):
                x_test.append(thumb)
                y_test.append(lion_class)

for i in range(0,np.shape(cut)[0],224):
    for j in range(0,np.shape(cut)[1],224):                
        thumb = cut[i:i+64,j:j+64,:]
        if np.amin(cv2.cvtColor(thumb, cv2.COLOR_BGR2GRAY)) != 0:
            if np.shape(thumb) == (64,64,3):
                x_test.append(thumb)
                y_test.append("negative") 
class_names.append("negative")
x_test = np.array(x_test)
y_test = np.array(y_test)
z_test = []
for img in x_test:
    img = np.array(img).reshape(12288)
    z_test.append(img)
z_test = np.array(z_test)

encoder = LabelBinarizer()
encoder.fit(y_test)
y_test = encoder.transform(y_test)

Predicting accuracy: The first value in the array of three is what the first runs accuracy had for output so I took that to be the main run and what I based my success on.  So, with the funs and the accuracy calculations I have strong evidence that the model is getting Females and Pups confused. Picture 8 below shows 2% accuracy. But if we inspect eh histogram below we see that most of the data is pup and females. Then in Picture 4 there was 87% accuracy for predicting classes and looking at the histogram below there were very few females and no pups present. This may be a coincidence and further study is needed but this is a very interesting trend.

In [ ]:
cvs = cross_val_score(clf, z_test, y_test, cv=3, scoring="accuracy")

In [ ]:
cvs

In [ ]:
index = 8
sl_counts = train_data.iloc[index]
print(sl_counts)

plt.figure()
sl_counts.plot(kind='bar', title='Count of Sea Lion Types')
plt.show()

In [ ]:
cvs = cross_val_score(clf, z_test, y_test, cv=3, scoring="accuracy")
cvs

In [ ]:
index = 4
sl_counts = train_data.iloc[index]
print(sl_counts)

plt.figure()
sl_counts.plot(kind='bar', title='Count of Sea Lion Types')
plt.show()

Frustrating to say the least: For the 4th picture I got 87% accuracy, 5th was 61%, 6th is 61%, 8th was 2%, 10th 81% and 12 was 81%. I really have no idea what to think about this. I believe finding the perfect distribution of data would be the best idea for trying to get a better model in my case.  Then I also believe that it is a valid point to say that pups and females are being confused and try to develop pictures where it is just isolated pups not have most of the pictures be a female and a pup. Then adding a lot of different negatives, there were all types of pictures with all types of topography and getting a good selection of different negatives would be ideal. The next goal would be to use a neural network and not a classifier like I did. I worked with Tensorflow for about 5 days then scrapped it because I wasn't getting anywhere. This was a bit of a baptism by fire into my first data science big project but I'm going to take what I learned and try and do better next time.   